In [3]:
import numpy as np
import gym
import torch

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningDataModule, Trainer

from gym.wrappers import RecordVideo, RecordEpisodeStatistics

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
from torch import nn

class dqn(nn.Module):
    def __init__(self, observation_dim, hidden_size, action_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(observation_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, action_dim)
        )

    def forward(self, observation):
        return self.net(observation)


In [5]:
def epsilon_policy(
    state, env, net, eplison=0.0, device="cuda" if torch.cuda.is_available() else "cpu"
):
    if np.random.random() < eplison:
        action = env.action_space.sample()
        return action
    else:
        state = torch.tensor([state]).to(device)
        q_values = net(state)
        _, action = torch.max(q_values, dim=1)
        # action = int(action.item())
        return q_values
    

In [6]:
import random

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

In [7]:
class RLDataset(torch.utils.data.dataset.IterableDataset):
    def __init__(self, buffer, sample_size=200):
        self.buffer = buffer
        self.sample_size = sample_size

    def __iter__(self):
        for experience in self.buffer.sample(self.sample_size):
            yield experience

In [8]:
def create_env(name):
    env = gym.make(name, render_mode="rgb_array")
    env = RecordVideo(env, video_folder = "./videos")
    return env
    

In [9]:
import matplotlib.pyplot as plt
env = create_env("LunarLander-v2")
env.reset()
frame = env.render()
plt.imshow(frame)

DependencyNotInstalled: box2D is not installed, run `pip install gym[box2d]`